In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from time import clock
%matplotlib inline

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

batch_size=16

# Make sure that caffe is on the python path:
caffe_root = '/home/lym/work/code/clcaffe-yolo/'  # this file is expected to be in {caffe_root}/examples
import os
os.chdir(caffe_root)
import sys
sys.path.insert(0, 'python')

import caffe
caffe.set_device(0)
caffe.set_mode_gpu()

In [ ]:
pic=[]
for i in range(0,batch_size):
    pic.append('examples/images/fish-bike.jpg')
pic[batch_size-1] = 'examples/images/cat.jpg'
#image = cv2.imread('examples/images/fish-bike.jpg')
image=[]
for i in range(0,batch_size):
    image.append( caffe.io.load_image(pic[i]) )
    
model_def = '/home/lym/work/code/clcaffe-yolo/models/yolo/yolo416/yolo_deploy_batch'+str(batch_size)+'.prototxt'
model_weights = '/home/lym/work/code/clcaffe-yolo/models/yolo/yolo416/yolo.caffemodel'

net = caffe.Net(model_def, model_weights, caffe.TEST)
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2, 0, 1))  # move image channels to outermost dimension
transformed_image=[None]*batch_size
for i in range(0,batch_size):
    transformed_image[i] = transformer.preprocess('data', image[i])
    
start = clock()
net.blobs['data'].reshape(batch_size, 3, 416, 416)
net.blobs['data'].data[...] = transformed_image
output = net.forward()
res = output['detection_out'][0]
stop = clock()
print res
print 'Executing Time of Prediction:',stop-start,'s'

index = 0
box = []
boxes = []
for c in range(res.shape[0]):
    for h in range(res.shape[1]):
        for w in range(res.shape[2]):
            box.append(res[c][h][w])
        boxes.append(box)
        box = []

w=[]
h=[]
im=[]
for i in range(0,batch_size):
    w.append(image[i].shape[1])
    h.append(image[i].shape[0])
    im.append(cv2.imread(pic[i]))

for box in boxes:
    left = (box[3]-box[5]/2.0) * w[(int)(box[0])];
    right = (box[3]+box[5]/2.0) * w[(int)(box[0])];
    top = (box[4]-box[6]/2.0) * h[(int)(box[0])];
    bot = (box[4]+box[6]/2.0) * h[(int)(box[0])];
    if left < 0:
        left = 0
    if right > w[(int)(box[0])]:
        right = w[(int)(box[0])]
    if top < 0:
        top = 0
    if bot > h[(int)(box[0])]:
        bot = h[(int)(box[0])]
    color = (255, 242, 35)
    cv2.rectangle(im[(int)(box[0])],(int(left), int(top)),(int(right),int(bot)),color,2)

for i in range(0,batch_size):
    plt.imshow(im[i])
    plt.show() 